# Netflix recommendation with python

In [26]:

pip install scikit-learn scipy matplotlib numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
#Import libraries

import pandas as pd
import numpy as np 
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity


In [28]:
#Read the dataset and display

data = pd.read_csv("netflixData.csv")
print(data.head(1))

                                Show Id     Title  \
0  cc1b6ed9-cf9e-4057-8303-34577fb54477  (Un)Well   

                                         Description Director      Genres  \
0  This docuseries takes a deep dive into the luc...      NaN  Reality TV   

  Cast Production Country  Release Date Rating  Duration Imdb Score  \
0  NaN      United States        2020.0  TV-MA  1 Season     6.6/10   

  Content Type Date Added  
0      TV Show        NaN  


In [29]:
data.columns

Index(['Show Id', 'Title', 'Description', 'Director', 'Genres', 'Cast',
       'Production Country', 'Release Date', 'Rating', 'Duration',
       'Imdb Score', 'Content Type', 'Date Added'],
      dtype='object')

In [30]:
print(data.isnull().sum())

Show Id                  0
Title                    0
Description              0
Director              2064
Genres                   0
Cast                   530
Production Country     559
Release Date             3
Rating                   4
Duration                 3
Imdb Score             608
Content Type             0
Date Added            1335
dtype: int64


##### The dataset contains null values, but before removing the null values, let’s select the columns that we can use to build a Netflix recommendation system:

In [33]:
data = data[["Title","Description","Content Type","Genres"]]
print(data.head(3))

                           Title  \
0                       (Un)Well   
1                         #Alive   
2  #AnneFrank - Parallel Stories   

                                         Description Content Type  \
0  This docuseries takes a deep dive into the luc...      TV Show   
1  As a grisly virus rampages a city, a lone man ...        Movie   
2  Through her diary, Anne Frank's story is retol...        Movie   

                                           Genres  
0                                      Reality TV  
1  Horror Movies, International Movies, Thrillers  
2             Documentaries, International Movies  


In [43]:
#Print shape and display null values

print(data.shape)

print(data.isnull().sum())

(5967, 4)
Title           0
Description     0
Content Type    0
Genres          0
dtype: int64


1. The title column contains the titles of movies and TV shows on Netflix
2. Description column describes the plot of the TV shows and movies
3. The Content Type column tells us if it’s a movie or a TV show
4. The Genre column contains all the genres of the TV show or the movie

### Now I will clean the Title column 


In [40]:
pip install nltk


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [47]:
import nltk 
import re
nltk.download ("stopwords")
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword=set(stopwords.words('english'))



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HellenMati\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
data["Title"] = data["Title"].apply(clean)

In [51]:
print(data.Title.sample(10))

868               calum von moger unbroken
2158                           husband goa
1784                                genius
1984                      hate dani rovira
3246                       mundina nildana
395                 apach life carlo tevez
2665                legendari weapon china
5798                              hero fli
2246                       inhuman resourc
4184    seven soul skull castl season wind
Name: Title, dtype: object


Now I will use the Genres column as the feature to recommend similar content to the user. I will use the concept of cosine similarity here (used to find similarities in two documents):

In [52]:
from spacy.lang.de.stop_words import STOP_WORDS

type(STOP_WORDS)

set

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.de.stop_words import STOP_WORDS
tfidf_text_vectorizer = TfidfVectorizer(stop_words=list(STOP_WORDS))


In [58]:
feature = data["Genres"].tolist()
tfidf = text.TfidfVectorizer(input=feature, stop_words="english")
tfidf_matrix = tfidf.fit_transform(feature)
similarity = cosine_similarity(tfidf_matrix)


InvalidParameterError: The 'input' parameter of TfidfVectorizer must be a str among {'content', 'filename', 'file'}. Got ['Reality TV', 'Horror Movies, International Movies, Thrillers', 'Documentaries, International Movies', 'TV Comedies', 'Documentaries, International Movies', 'Dramas, International Movies, Romantic Movies', 'Dramas, International Movies, Romantic Movies', 'Comedies', 'Documentaries, Sports Movies', 'Comedies, Dramas, International Movies', 'Comedies, Dramas, International Movies', 'Comedies, International Movies, Romantic Movies', 'Comedies, Dramas, International Movies', 'International TV Shows, Romantic TV Shows, TV Dramas', 'Docuseries, Science & Nature TV', 'Dramas, International Movies, Sports Movies', 'Movies', 'Dramas, International Movies', 'Dramas, International Movies', 'Horror Movies, International Movies', 'Crime TV Shows, TV Dramas, TV Mysteries', 'Crime TV Shows, Docuseries', 'Documentaries', 'Documentaries', 'Comedies, Dramas, Independent Movies', 'Dra

Now I will set the Title column as an index so that we can find similar content by giving the title of the movie or TV show as an input:

In [59]:
def netFlix_recommendation(title, similarity = similarity):
    index = indices[title]
    similarity_scores = list(enumerate(similarity[index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[0:10]
    movieindices = [i[0] for i in similarity_scores]
    return data['Title'].iloc[movieindices]

print(netFlix_recommendation("girlfriend"))

NameError: name 'similarity' is not defined

### The recommendation system of Netflix predicts a personalised catalogue for you based on factors like your viewing history, the viewing history of other users with similar tastes and preferences, and the genres, category, descriptions, and more information of the content you watched.